In [2]:
# !pip install pytorch-lightning
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from torchmetrics import Accuracy
from datasets import load_dataset
import numpy as np
import pandas as pd

# ignore warning
import warnings

warnings.filterwarnings("ignore")

In [6]:
class MultiLayerPerceptron(pl.LightningModule):
    def __init__(self, image_shape=(1, 28, 28), hidden_units=(32, 16), num_classes=10):
        super().__init__()

        # new PL attributes:
        self.train_acc = Accuracy(task="multiclass", num_classes=num_classes)
        self.valid_acc = Accuracy(task="multiclass", num_classes=num_classes)
        self.test_acc = Accuracy(task="multiclass", num_classes=num_classes)

        input_size = image_shape[0] * image_shape[1] * image_shape[2]
        all_layers = [nn.Flatten()]
        for hidden_unit in hidden_units:
            layer = nn.Linear(input_size, hidden_unit)
            all_layers.append(layer)
            all_layers.append(nn.ReLU())
            input_size = hidden_unit

        all_layers.append(nn.Linear(hidden_units[-1], 10))
        self.model = nn.Sequential(*all_layers)

    def forward(self, x):
        x = self.model(x)
        return x

In [7]:
def training_step(self, batch, batch_idx):
    x, y = batch
    logits = self(x)
    loss = nn.functional.cross_entropy(logits, y)
    preds = torch.argmax(logits, dim=1)
    self.train_acc.update(preds, y)
    self.log("train_loss", loss, prog_bar=True)
    return loss


def training_epoch_end(self, outs):
    self.log("train_acc", self.train_acc.compute())
    self.train_acc.reset()


def validation_step(self, batch, batch_idx):
    x, y = batch
    logits = self(x)
    loss = nn.functional.cross_entropy(logits, y)
    preds = torch.argmax(logits, dim=1)
    self.valid_acc.update(preds, y)
    self.log("valid_loss", loss, prog_bar=True)
    return loss


def validation_epoch_end(self, outs):
    self.log("valid_acc", self.valid_acc.compute(), prog_bar=True)
    self.valid_acc.reset()


def test_step(self, batch, batch_idx):
    x, y = batch
    logits = self(x)
    loss = nn.functional.cross_entropy(logits, y)
    preds = torch.argmax(logits, dim=1)
    self.test_acc.update(preds, y)
    self.log("test_loss", loss, prog_bar=True)
    self.log("test_acc", self.test_acc.compute(), prog_bar=True)
    return loss


def configure_optimizers(self):
    optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
    return optimizer

In [10]:
pip install --upgrade --force-reinstall numpy

  Using cached numpy-1.26.2-cp39-cp39-win_amd64.whl (15.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.0
    Uninstalling numpy-1.25.0:
      Successfully uninstalled numpy-1.25.0
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.6.0 requires daal==2021.4.0, which is not installed.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.26.2 which is incompatible.
tensorboard 2.10.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.1 which is incompatible.
scipy 1.9.1 requires numpy<1.25.0,>=1.18.5, but you have numpy 1.26.2 which is incompatible.
octis 1.11.0 requires scikit-learn==0.24.2, but you have scikit-learn 1.2.2 which is incompatible.
en-core-web-sm 3.4.0 requires spacy<3.5.0,>=3.4.0, but you have spacy 3.6.0 which is incompatible.


In [9]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

ImportError: DLL load failed while importing lapack_lite: The specified module could not be found.

In [5]:
class MyDataset(Dataset):
    def __init__(self, dataset, transform=None):
        self.images = dataset["img"]
        self.labels = dataset["label"]
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        if self.transform:
            augmented = self.transform(image=np.array(image))
            image = augmented["image"]
        return {
            "image": image,
            "label": label,
        }

In [6]:
# define reusable config
CONFIG = dict(
    batch_size=2,
    img_size=32,
    hidden_units=(32, 16),
    max_epochs=10,
    gpus=1 if torch.cuda.is_available() else 0,
)


class MnistDataModule(pl.LightningDataModule):
    def __init__(self, batch_size):
        super().__init__()
        self.batch_size = batch_size
        # Train augmentation policy
        self.train_transform = A.Compose(
            [
                A.RandomResizedCrop(
                    height=CONFIG["img_size"], width=CONFIG["img_size"]
                ),
                A.HorizontalFlip(p=0.5),
                A.ShiftScaleRotate(p=0.5),
                A.RandomBrightnessContrast(p=0.5),
                A.Normalize(),
                ToTensorV2(),
            ]
        )

        # Validation/Test augmentation policy
        self.test_transform = A.Compose(
            [
                A.Resize(height=CONFIG["img_size"], width=CONFIG["img_size"]),
                A.Normalize(),
                ToTensorV2(),
            ]
        )

    def prepare_data(self):
        self.train_ds, self.test_ds = load_dataset(
            "cifar10", split=["train[:1000]", "test[:200]"]
        )

    def setup(self, stage=None):
        # stage is either 'fit', 'validate', 'test', or 'predict'
        self.train = MyDataset(self.train_ds, self.train_transform)
        self.valid = MyDataset(self.test_ds, self.test_transform)
        self.test = MyDataset(self.test_ds, self.test_transform)

    def train_dataloader(self):
        return DataLoader(
            self.train,
            batch_size=self.batch_size,
            shuffle=True,
        )

    def val_dataloader(self):
        return DataLoader(
            self.valid,
            batch_size=self.batch_size,
        )

    def test_dataloader(self):
        return DataLoader(
            self.test,
            batch_size=self.batch_size,
        )

In [7]:
datamodule = MnistDataModule(batch_size=CONFIG["batch_size"])
datamodule.prepare_data()
datamodule.setup()
# Samples required by the custom ImagePredictionLogger callback to log image predictions.
val_samples = next(iter(datamodule.val_dataloader()))
val_imgs, val_labels = val_samples["image"], val_samples["label"]
val_imgs.shape, val_labels.shape

Found cached dataset cifar10 (C:/Users/soiko/.cache/huggingface/datasets/cifar10/plain_text/1.0.0/447d6ec4733dddd1ce3bb577c7166b986eaa4c538dcd9e805ba61f35674a9de4)


  0%|          | 0/2 [00:00<?, ?it/s]

(torch.Size([2, 3, 32, 32]), torch.Size([2]))

In [8]:
!conda list numpy

# packages in environment at C:\Users\soiko\anaconda3:
#
# Name                    Version                   Build  Channel
numpy                     1.26.2                   pypi_0    pypi
numpy-base                1.21.5           py39hca35cd5_3  
numpydoc                  1.4.0            py39haa95532_0  
